# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
# Store csv created in part one into a DataFrame
weather_data_df = pd.read_csv("output_data/cities.csv")
weather_data_df.head()
weather_data_df.columns

Index(['City_ID', 'City', 'Cloudiness', 'Country', 'Date', 'Humidity', 'Temp',
       'Lat', 'Lng', 'Wind Speed'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [30]:
# Store latitude and longitude in locations
lat_lon_locations = weather_data_df[['Lat', 'Lng']]

# Fill NaN values and convert to float
humidity = weather_data_df["Humidity"]
# lat_lon.head()
# humidity.head()

# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays 
# properly at a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lon_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)
# Add heat layer
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned 
# by your API requests to a reasonable number.

temperature_criteria = (weather_data_df['Temp'] < 80) & (weather_data_df.Temp > 70)
wind_speed_criteria = weather_data_df['Wind Speed'] < 10
cloudiness_criteria = weather_data_df['Cloudiness'] == 0
final_criteria = temperature_criteria & wind_speed_criteria & cloudiness_criteria

# Use boolean indexing to filter the weather_data_df dataframe to drop anything that doesn't fit the three criterias
ideal_weather_df = weather_data_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,45,45,luocheng,0,CN,1585633076,31,73.02,29.38,104.03,4.90
1,56,56,cidreira,0,BR,1585632805,89,71.47,-30.18,-50.21,0.58
2,118,118,xichang,0,CN,1585633092,14,78.04,27.90,102.26,5.86
3,196,196,gwadar,0,PK,1585632951,77,75.56,25.12,62.33,5.46
4,252,252,sur,0,OM,1585632906,66,78.55,22.57,59.53,9.80
5,270,270,puerto escondido,0,MX,1585632865,64,78.42,15.85,-97.07,4.29
6,414,414,imbituba,0,BR,1585632902,83,77.00,-28.24,-48.67,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
# Store into variable named hotel_df
# Add a "Hotel Name" column to the DataFrame.

hotel_df = ideal_weather_df
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Hotel Name
0,45,45,luocheng,0,CN,1585633076,31,73.02,29.38,104.03,4.90,
1,56,56,cidreira,0,BR,1585632805,89,71.47,-30.18,-50.21,0.58,
2,118,118,xichang,0,CN,1585633092,14,78.04,27.90,102.26,5.86,
3,196,196,gwadar,0,PK,1585632951,77,75.56,25.12,62.33,5.46,
4,252,252,sur,0,OM,1585632906,66,78.55,22.57,59.53,9.80,


In [45]:
# Set parameters to search for hotels with 5000 meters.

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Iterate through the rows to us the lat/long to get the hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location for each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.info()

Missing field/result... skipping.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 12 columns):
index         7 non-null int64
City_ID       7 non-null int64
City          7 non-null object
Cloudiness    7 non-null int64
Country       7 non-null object
Date          7 non-null int64
Humidity      7 non-null int64
Temp          7 non-null float64
Lat           7 non-null float64
Lng           7 non-null float64
Wind Speed    7 non-null float64
Hotel Name    7 non-null object
dtypes: float64(4), int64(5), object(3)
memory usage: 800.0+ bytes


In [47]:
hotel_df.head()

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed,Hotel Name
0,45,45,luocheng,0,CN,1585633076,31,73.02,29.38,104.03,4.90,
1,56,56,cidreira,0,BR,1585632805,89,71.47,-30.18,-50.21,0.58,Hotel Castelo
2,118,118,xichang,0,CN,1585633092,14,78.04,27.90,102.26,5.86,Meili Shanshui Hotel
3,196,196,gwadar,0,PK,1585632951,77,75.56,25.12,62.33,5.46,Sadaf Resort
4,252,252,sur,0,OM,1585632906,66,78.55,22.57,59.53,9.80,Sur Grand Hotel


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

# Display map
fig

# Add marker layer ontop of heat map
# marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(marker_layer)

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…